In [124]:
import json
import requests
import pandas as pd
import numpy as np
import math
import scrapy
from azure.cognitiveservices.search.newssearch import NewsSearchAPI
from msrest.authentication import CognitiveServicesCredentials

In [10]:
url = ('https://newsapi.org/v2/top-headlines?'
       'country=us&'
       'apiKey=0268f5ffc11b4e3e85a9e6193281aa81')
response = requests.get(url)
data = response.json()

In [15]:
json.dumps(data)

'{"status": "ok", "totalResults": 38, "articles": [{"source": {"id": "the-washington-post", "name": "The Washington Post"}, "author": "Colby Itkowitz", "title": "Group that awarded Trump removed from criminal justice event over Kamala Harris boycott - The Washington Post", "description": "The California senator and presidential candidate accused the event organizers of \\u201cpapering over his record.\\u201d", "url": "https://www.washingtonpost.com/politics/trump-slams-kamala-harris-for-skipping-criminal-justice-event-that-awarded-him/2019/10/26/01e2b406-f7fb-11e9-8cf0-4cc99f74d127_story.html", "urlToImage": "https://www.washingtonpost.com/resizer/fHnxSv3XurkrlwvNXYXrMucJaY0=/1440x0/smart/arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/OLZVOKXYA4I6TDHQJTEZ65GRE4.jpg", "publishedAt": "2019-10-26T16:59:00Z", "content": "Donald Trump is a lawless President. Not only does he circumvent the laws of our country and the principles of our Constitution, but there is nothing in his

In [18]:
for item in data:
    print(item)
articles = data['articles']


status
totalResults
articles
ok


In [ ]:
with open('data.json', 'w') as file:
    json.dump(data, file)

In [242]:
subscription_key = '2db741825a6e49b9b86bc63d41337bcc'
search_term = "shaken baby syndrome california"
client = NewsSearchAPI(CognitiveServicesCredentials(subscription_key))
news_result = client.news.search(query=search_term, market="en-us", count=1000)

In [245]:
titles = []
descriptions = []
for i in range(len(news_result.value)):
    article = news_result.value[i]
    titles.append(article.name)
    descriptions.append(article.description)
print(titles[0])
print(descriptions[0])
print(news_result.value[0].url)

Merced father accused in shaken baby syndrome case
Doctors at Children's Hospital Central California later determined the infant's injuries did not appear accidental and more in line with shaken baby syndrome. The baby was declared brain-dead over the weekend.
https://abc13.com/archive/8351533/


In [246]:
if news_result.value:
    first_news_result = news_result.value[0]
    print("Total estimated matches value: {}".format(
        news_result.total_estimated_matches))
    print("News result count: {}".format(len(news_result.value)))
    print("First news name: {}".format(first_news_result.name))
    print("First news url: {}".format(first_news_result.url))
    print("First news description: {}".format(first_news_result.description))
    print("First published time: {}".format(first_news_result.date_published))
    print("First news provider: {}".format(first_news_result.provider[0].name))
else:
    print("Didn't see any news result data..")

Total estimated matches value: 98
News result count: 98
First news name: Merced father accused in shaken baby syndrome case
First news url: https://abc13.com/archive/8351533/
First news description: Doctors at Children's Hospital Central California later determined the infant's injuries did not appear accidental and more in line with shaken baby syndrome. The baby was declared brain-dead over the weekend.
First published time: 2019-10-09T00:00:00.0000000Z
First news provider: abc13


In [247]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N/ float(val))
    return idfDict

In [248]:
def allUniqueWords(docList):
    uniq = []
    for text in docList:
        terms = text.split()
        for word in terms:
            word = word.lower()
            if word not in uniq:
                uniq.append(word)
    return uniq
unique = allUniqueWords(descriptions)
print(len(unique))

1000


In [249]:
def allPronoun(text):
    pro = []
    terms = text.split()
    for word in terms:
        if(not word.islower):
            pro.append(word)
    return pro[0,3]

In [250]:
def computeTF(docList, unique):
    data = pd.DataFrame(columns=unique)
    for i in range(len(docList)):
        data.loc[i] = np.zeros(len(unique))
    row = 0
    col = 0
    for text in docList:
        terms = text.split()
        for word in terms:
            word = word.lower()
            data.loc[row, word] += 1 
        row += 1
    return data
whole = computeTF(descriptions, unique)
print(whole)

    doctors   at  children's  hospital  central  california  later  \
0       1.0  1.0         1.0       1.0      1.0         1.0    1.0   
1       0.0  0.0         0.0       0.0      0.0         1.0    0.0   
2       0.0  1.0         0.0       0.0      0.0         1.0    0.0   
3       0.0  0.0         0.0       0.0      0.0         1.0    0.0   
4       0.0  0.0         0.0       0.0      0.0         1.0    0.0   
..      ...  ...         ...       ...      ...         ...    ...   
93      0.0  0.0         0.0       0.0      0.0         0.0    0.0   
94      0.0  0.0         0.0       0.0      0.0         1.0    0.0   
95      0.0  0.0         0.0       0.0      0.0         1.0    0.0   
96      0.0  0.0         0.0       0.0      0.0         1.0    0.0   
97      0.0  0.0         0.0       0.0      0.0         1.0    0.0   

    determined  the  infant's  ...  consensus  evolved  2002  attracted  \
0          1.0  3.0       1.0  ...        0.0      0.0   0.0        0.0   
1        

In [251]:
len(whole.columns)

1000

In [252]:
def computeTFIDF(TF, unique):
    doc_data = pd.DataFrame(columns=unique)
    doc_data.loc[0] = np.zeros(len(unique))
    #print(doc_data)
    N = TF.shape[0]
    for i in range(N):
        for word in whole.columns:
            if (TF.loc[i, word] > 0):
                doc_data.loc[0, word] += 1
    for word in doc_data.columns:
        doc_data.loc[0, word] = math.log10(N / (1 + doc_data.loc[0, word]))
    for word in TF.columns:
        for i in range(N):
            TF.loc[i, word] = TF.loc[i, word] * doc_data.loc[0,word]
    return TF

In [253]:
TFIDF = computeTFIDF(whole, unique)
TFIDF

,doctors,at,children's,hospital,central,california,later,determined,the,infant's,...,consensus,evolved,2002,attracted,attention,project.,university-based,group,founded,same
0,1.292256,0.411442,1.690196,1.389166,1.389166,0.056728,1.389166,1.514105,0.314206,1.292256,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.056728,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.411442,0.000000,0.000000,0.000000,0.056728,0.000000,0.000000,0.104735,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.056728,0.000000,0.000000,0.314206,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.056728,0.000000,0.000000,0.314206,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.104735,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
94,0.000000,0.000000,0.000000,0.000000,0.000000,0.056728,0.000000,0.000000,0.418941,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
95,0.000000,0.000000,0.000000,0.000000,0.000000,0.056728,0.000000,0.000000,0.104735,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
96,0.000000,0.000000,0.000000,0.000000,0.000000,0.056728,0.000000,0.000000,0.104735,0.000000,...,1.690196,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [255]:
row = TFIDF.loc[0,:]
print(row[0])

1.2922560713564761


In [256]:
def Nmaxelements(list1, N): 
    final_list = [] 
  
    for i in range(0, N):  
        max1 = 0
        maxi = -1
          
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j] 
                maxi = j 
        list1.remove(max1); 
        final_list.append(maxi) 
    return(final_list)
    print(final_list)

In [266]:
keywords = [[0 for x in range(3)] for x in range(TFIDF.shape[0])]
for index in range(TFIDF.shape[0]):
    row = TFIDF.loc[index,:]
    max_index = Nmaxelements(row.tolist(), 3)
    #result[index] = allPronoun(descriptions[index])
    for i in max_index:
        keywords[index] = (TFIDF.columns[i])
keywords

['did',
 'child',
 'infant',
 'ruling',
 'sentence',
 'already',
 'northwestern',
 'oklahoma',
 '"shaken',
 'set',
 'house',
 'set',
 '“both',
 'who',
 'who',
 'nationwide',
 'kaneb,',
 'house',
 'california:',
 'who',
 'bridget',
 'who',
 '[caption',
 'which',
 'los',
 'under',
 'could',
 'retry',
 'syndrome”',
 'conviction,',
 'transported',
 'shakes,',
 'passed',
 'belief',
 'there',
 'search',
 'syndrome”',
 '--',
 'syndromemaintains',
 'edward',
 'jan.',
 'jury',
 'young',
 'passed',
 '"we\'ve',
 'would',
 'certainty',
 'as',
 'bransen',
 'california.',
 'passed',
 'berkeley',
 'passed',
 'johan',
 'california.',
 'affidavit',
 "ribe's",
 'forefront',
 'reuters:',
 'belief',
 'francisco-based',
 'necessarily',
 'belief',
 'injury,',
 'slammed',
 'slammed',
 'washington,',
 "ribe's",
 'protracted',
 "ribe's",
 'underestimate',
 'fought',
 'physical',
 'cases”',
 'december',
 '“you',
 "ribe's",
 'always',
 'abuse',
 'only',
 'warrant',
 'effective',
 'sample',
 'necessarily',
 'poli